<h2 dir="rtl">
در این ثسمت میخواهیم که پروژه تشخیص قصد با استفاده از Naive Baise بر روی وکتورهای tf-idf به زبان فارسی پیاده را پیاده کنیم.


</h2>

<h4 dir="rtl">
کتابخانه‌های مورد نیاز برای دیتاست‌ها، ارزیابی‌ها، پردازش متنی به زبان فارسی، مدل‌های متنی، و... برای آموزش و اجرای مدل‌های متنی هستند.
</h4>

In [1]:
!pip install datasets
!pip install -q datasets
!pip install -q evaluate
!pip install -q --upgrade hazm
!pip install -q sentencepiece
!pip install -q torch
!pip install -q rich[jupyter]
!pip install -q datasets


<h3 dir="rtl">
 در کد زیر هم کتابخانه های مورد نظر را فرا خواندیم که در ادامه شرح میدهیم که هر یک چه کار میکنند

  
 numpy  و skearn برای محاسبات علمی و یادگیری ماشین به کار گرفته می‌شوند.


 از TfidfVectorizer در scikit-learn برای تبدیل متن‌ها به بردارهای TF-IDF استفاده می‌شود. این بردارها ویژگی‌های متن را با توجه به میزان تکرار و مهمیت کلمات در متن محاسبه می‌کنند.


 train_test_split  برای تقسیم مجموعه داده به دو بخش آموزش و آزمون به نسبت مشخصی به کار می‌رود.


 classification_report  برای ارزیابی عملکرد مدل بر روی داده‌های آزمون با استفاده از معیارهای دقت، بازیابی، و امتیاز F1 به کار می‌رود.


از مدل MultinomialNB در scikit-learn به عنوان مدل یادگیری ماشین برای تشخیص اندازه‌گیری (Intent Detection) استفاده می‌شود. این مدل از روش بیز ساده برای پیش‌بینی دسته‌های مختلف بر اساس بردارهای TF-IDF استفاده می‌کند.


load_dataset از کتابخانه Hugging Face Datasets برای بارگذاری دیتاست مورد استفاده در پروژه به کار می‌رود.

</h3>

In [2]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from datasets import load_dataset

In [3]:
!pip install datasets

In [4]:
from datasets import load_dataset

<h3 dir="rtl">


 از دستور load_dataset("AmazonScience/massive", 'fa-IR')  برای بارگیری مجموعه داده استفاده می‌کنیم.


 dataset = dataset.rename_column("utt", "text") در این قسمت، نام ستون "utt" (متن داده) به "text" تغییر میدیم.


 dataset = dataset.rename_column("scenario", "label")`  نام ستون "scenario" (برچسب‌ها) به "label" تغییر میدهیم.  


  در قسمت آخر هم داده‌های آموزش و آزمون از مجموعه داده به دست می‌آید.
</h3>

In [5]:
dataset = load_dataset("AmazonScience/massive", 'fa-IR')
dataset = dataset.rename_column("utt", "text")
dataset = dataset.rename_column("scenario", "label")

# Access the text and label columns based on the keys
train_texts = dataset['train']['text']
train_labels = dataset['train']['label']
test_texts = dataset['test']['text']
test_labels = dataset['test']['label']


<h3 dir="rtl">
 در ادامه میخواهیم که تبدیل متن‌ها به بردارهای عددی را در جهت استفاده در مدل‌های یادگیری ماشینی استفاده کنیم.



در ابتدا یک شیء از کلاس TF-IDF Vectorizer ایجاد می‌کنیم.
max_features که بیانگر تعداد حداکثر کلماتی که به عنوان ویژگی‌ها در نظر گرفته می‌شوند را ۵۰۰۰ قرار میدهیم.
encoding و کدگذاری متن‌ها دا به صورت 'utf-8' مشخص میکنیم.
سپس، با استفاده از fit_transform، متن‌های آموزش (train_texts) به بردارهای TF-IDF تبدیل می‌کینم و در train_tfidf ذخیره می‌کینم. این بردارها شامل اطلاعات مرتبط با فراوانی واژه‌ها در متن‌های آموزش هستند.



</h3>

In [6]:
# TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words=None, encoding='utf-8')
train_tfidf = tfidf_vectorizer.fit_transform(train_texts)
test_tfidf = tfidf_vectorizer.transform(test_texts)

<h3 dir="rtl">
در ادامه میخواهیم تبدیل برچسب‌های دسته‌بندی شده به اعداد صحیح را انجام بدیم.


 یک نمونه از کلاس  LabelEncoder ایجاد می‌کنیم. سپس fit_transform برای آموزش و تبدیل برچسب‌های آموزشی (train_labels) به اعداد صحیح استفاده می‌شود. این عمل با تبدیل هر برچسب به یک عدد منحصر به فرد انجام می‌شود.
 transform همچنین برای تبدیل برچسب‌های تستی (test_labels) به اعداد صحیح استفاده می‌شود.



</h3>

In [7]:
# Encode labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

<h3 dir="rtl">
در ادامه از الگوریتم MultinomialNB استفاده می‌کنیم که برای مسائل طبقه‌بندی چند کلاسه معمولاً مناسب است.

 naive_bayes_classifier = MultinomialNB() : ایجاد یک نمونه از مدل کلاس‌بندی بیزی چند جمله‌ای ایجاد مکنیم.
سپس مدل  را با استفاده از داده‌های آموزش train_tfidf و برچسب‌های متناظر با آنها   train_labels_encoded اوکی میکنیم که train_tfidf  حاوی بردارهای TF-IDF مربوط به متن‌های آموزش است و  train_labels_encoded حاوی برچسب‌های عددی متناظر با کلاس‌های مختلف مسئله است.



</h3>

In [8]:
# Train Naive Bayes classifier
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(train_tfidf, train_labels_encoded)

MultinomialNB()

<h3 dir="rtl">
حال به سراغ  پیش‌بینی برچسب‌ها بر روی مجموعه آزمون و  بازگرداندن برچسب‌های پیش‌بینی شده به شکل اصلی میرویم.

در ابتدا مدل ماشین یادگیری Naive Bayes برای پیش‌بینی برچسب‌های کلاس‌ها بر روی داده‌های تست استفاده می‌شود. به عبارت دیگر، ما داریم پیشبینی میکنیم که هر جمله تست به کدام کلاس از کلاس‌ها مپ می‌شود. سپس برچسب‌هایی که توسط مدل پیش‌بینی شده‌اند (به صورت اعداد) به تبدیل به برچسب‌های اصلی (رشته‌ها) می‌پردازیم.


</h3>

In [9]:
# Predict labels on the test set
predicted_labels = naive_bayes_classifier.predict(test_tfidf)

# Decode labels
predicted_labels_decoded = label_encoder.inverse_transform(predicted_labels)

<h3 dir="rtl">



کد ابتدا از ماژول sklearn.metrics تابع classification_report را فراخوانی می‌کند. سپس این تابع بر اساس پیش‌بینی‌های مدل predicted_labels_decoded  و برچسب‌های واقعی test_labels ، یک گزارش ارزیابی ایجاد می‌کند.  



</h3>

In [10]:
# Evaluate the classifier
classification_rep = classification_report(test_labels, predicted_labels_decoded, labels=np.unique(predicted_labels_decoded))
print(classification_rep)


              precision    recall  f1-score   support

           0       1.00      0.52      0.68       106
           1       0.94      0.89      0.91       124
           2       0.49      0.99      0.65       402
           3       0.79      0.99      0.88       387
           4       0.97      0.74      0.84       124
           5       1.00      0.53      0.70       103
           6       0.94      0.53      0.68        94
           7       0.81      0.94      0.87       271
           8       0.96      0.94      0.95       220
           9       0.77      0.23      0.35       189
          10       1.00      0.52      0.68        62
          11       0.88      0.53      0.66       142
          12       0.75      0.85      0.79       288
          13       0.96      0.35      0.51        72
          14       1.00      0.61      0.76        57
          15       1.00      0.31      0.47        81
          16       0.98      0.48      0.64        96
          17       0.95    

<h3 dir="rtl">


در ادامه هم جمله ورودی میدیم و خروجی را مشاهده میکنیم.
منتها خروجی ما به صودت عدد است به خاطر نوع دیتاست مان.

</h3>

In [11]:
# Example of predicting the intent of a new sentence
new_sentence = "گوشی هوشمند من نیازمند ماژول است"
new_sentence_tfidf = tfidf_vectorizer.transform([new_sentence])
predicted_intent = naive_bayes_classifier.predict(new_sentence_tfidf)
predicted_intent_decoded = label_encoder.inverse_transform(predicted_intent)
print(f"Predicted intent for '{new_sentence}': {predicted_intent_decoded[0]}")

Predicted intent for 'گوشی هوشمند من نیازمند ماژول است': 8


<h3 dir="rtl">


در ادامه یک تابع به نام map_number_to_word ایجاد می‌کنیم که عددی و یک آرایه از کلمات به عنوان ورودی می‌گیرد. سپس اگر عدد در محدوده صحیح واقعی بین ۰ و تعداد کلمات در آرایه باشد، تابع کلمه متناظر با این عدد را برمی‌گرداند.


</h3>

In [12]:
def map_number_to_word(number, word_array):
    if 0 <= int(number) < len(word_array):
        return word_array[int(number)]
    else:
        return "Number out of range"


word_array = [
    'social', 'transport', 'calendar', 'play', 'news', 'datetime',
    'recommendation', 'email', 'iot', 'general', 'audio', 'lists',
    'qa', 'cooking', 'takeaway', 'music', 'alarm', 'weather'
]


<h3 dir="rtl">

در نهایت خروجی مان به صودت موضوعی مشخص میشود


</h3>

In [13]:
result = map_number_to_word(predicted_intent_decoded[0], word_array)
print(f"Number {int(predicted_intent_decoded[0])} means : '{result}'")


Number 8 means : 'iot'


<h3 dir="rtl">
مقایسه:



  عملکرد (Performance)
   - روش TF-IDF و Naive Bayes: این روش برای مجموعه داده‌های کوچک تا متوسط ​​بسیار سریع و دقیق عمل می‌کند. اگر تعداد برچسب‌ها و تعداد واژه‌های مجموعه داده کم باشد، این روش به عنوان یک گزینه مناسب مطرح می‌شود.
   - روش مبتنی بر ترنسفورمر: ترنسفورمرها، به ویژه مدل‌های بزرگتر، برای مجموعه داده‌های بزرگ و پیچیده عملکرد بهتری دارند. آن‌ها می‌توانند از اطلاعات متنی بهتری برای دسته‌بندی استفاده کنند و از انتقال یادگیری نیز بهره‌مند شوند.

 کارایی (Efficiency):
   - روش TF-IDF و Naive Bayes: این روش در مقایسه با ترنسفورمرها کمترین نیاز به منابع محاسباتی دارد. بنابراین، از نظر کارایی و سرعت، این روش ممکن است بهتر باشد.
   - روش مبتنی بر ترنسفورمر: ترنسفورمرها نیاز به مدل‌های بزرگ و محاسبات بیشتر دارند. این ممکن است منابع بیشتری را مصرف کند و زمان زیادی برای آموزش و پیش‌بینی‌ها در مقایسه با TF-IDF و Naive Bayes ببرد.

 حافظه (Memory):
   - روش TF-IDF و Naive Bayes: این روش به حافظه کمتری نیاز دارد. حجم ماتریس TF-IDF به تعداد واژه‌های موجود در متن و تعداد نمونه‌ها بستگی دارد.
   - روش مبتنی بر ترنسفورمر: ترنسفورمرها به حافظه بیشتری نیاز دارند، به ویژه مدل‌های بزرگ. حجم مدل‌های ترنسفورمر معمولاً بزرگتر است و ممکن است به حافظه بیشتری نیاز داشته باشد.

 امکانات (Capabilities):
   - روش TF-IDF و Naive Bayes: این روش برای مجموعه داده‌های کم تنوع، محدود به واژگان موجود در متن‌هاست و نیاز به پیش‌پردازش دقیق دارد. نیاز به تنظیمات دقیق‌تر است.
   - روش مبتنی بر ترنسفورمر: ترنسفورمرها به نسبت بهتری می‌توانند با متون گوناگون و معناهای مختلف کار کنند و نیاز به پیش‌پردازش کمتری دارند. آموزش و تطابق بهتری با داده‌ها دارند.

</h3>

<h3 dir="rtl">
نتیجه گیری:



بنابراین، اگر مجموعه داده   کوچک و ساده است و توانایی پردازش مصرف حافظه و محدودیت منابع محاسباتی را دارید، روش TF-IDF و Naive Bayes ممکن است بهترین گزینه باشد. اگر مجموعه داده   بزرگتر و پیچیده‌تر است و توانایی ایجاد و استفاده از مدل‌های بزرگتر و پیچیده‌تر را دارید، روش مبتنی بر ترنسفورمر ممکن است بهترین انتخاب باشد.



</h3>